In [1]:
!pip install --quiet /kaggle/input/kerasapplications
!pip install --quiet /kaggle/input/efficientnet-git

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
import cv2
from tensorflow.keras.layers import Dense
import os
import efficientnet.tfkeras as efn
from tqdm import tqdm as tqdm
import numpy as np

# Loading in model

In [3]:
IMG_SIZE = 256
def make_model():
    model = Sequential()
    model.add(efn.EfficientNetB4(include_top = False,
                                  weights = None,
                                  pooling = 'avg',
                                  input_shape = (IMG_SIZE, IMG_SIZE, 3)))
    model.add(Dense(5, activation = 'softmax'))
    model.layers[0].trainable = True
    
#     optimizer = Adam(lr = 0.01)
#     #tpu_optimizer = tf.experimental.contrib.tpu.CrossShardOptimizer(optimizer)
#     model.compile(optimizer=optimizer, 
#                   loss='categorical_crossentropy', 
#                   metrics=['accuracy'])
    return model

In [4]:
model = make_model()
model.load_weights('../input/v0-1-cassava-efficientnet-training/EnetB4_top_trainable/best_model.hdf5')

# Data Augmentation Function - 

In [5]:
def data_augment(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
            
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90º
        
    # Pixel-level transforms

    if p_pixel_1 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
            
    return image

# Look at the sample submission file

In [6]:
pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

,image_id,label
0,2216849948.jpg,4


In [7]:
sub_dict = {'image_id':[], 'label':[]}

In [8]:
test_data_path = '../input/cassava-leaf-disease-classification/test_images/'
for img_path in tqdm(os.listdir(test_data_path)):
    img = cv2.imread(test_data_path + img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = np.array(data_augment(img))
    prediction = np.argmax(model.predict([img.reshape(-1, IMG_SIZE, IMG_SIZE, 3)]))
    sub_dict['image_id'].append(img_path)
    sub_dict['label'].append(prediction)
    

100%|██████████| 1/1 [00:05<00:00,  5.78s/it]


In [9]:
submission_df = pd.DataFrame(sub_dict)
submission_df.head()

,image_id,label
0,2216849948.jpg,0


In [10]:
submission_df.to_csv("submission.csv", index=False)